# 08-01-Milestone-Project-2


Assignment: Create a Complete BlackJack Card Game in Python.

Here are the requirements:

- You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
- The game needs to have one player versus an automated dealer.
- The player can stand or hit.
- The player must be able to pick their betting amount.
- You need to keep track of the player's total money.
- You need to alert the player of wins, losses, or busts, etc...
- You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand.


___
# Solution

## Deck class

In [73]:
%%writefile "programs/08-Milestone-Project-2/Deck_class.py"
# imports
import random

# definitions
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven',
         'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
# definitions
values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7,
          'Eight': 8, 'Nine': 9, 'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}
          
class Deck:

    def __init__(self):
        self.all_cards = []

        # Use Card class to populate deck
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit, rank))

    def shuffle(self):
        random.shuffle(self.all_cards)

    def deal(self):
        return self.all_cards.pop()

    def __str__(self):
        card_list = [str(card) for card in self.all_cards]
        return f"[{' | '.join(card_list)}]"


class Card:

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank} of {self.suit}"

Overwriting programs/08-Milestone-Project-2/Deck_class.py


### Test Deck class

In [74]:
%%writefile "programs/08-Milestone-Project-2/test_Deck_class.py"
# imports
import unittest
from Deck_class import *

class TestDeckAndCard(unittest.TestCase):
    def setUp(self):
        # Crie uma instância de Deck para usar nos testes
        self.deck = Deck()

    def test_deck_initialization(self):
        # Verifique se o deck foi inicializado corretamente
        self.assertEqual(len(self.deck.all_cards), 52)

    def test_deck_shuffle(self):
        # Verifique se o embaralhamento altera a ordem das cartas
        original_order = self.deck.all_cards[:]
        self.deck.shuffle()
        self.assertNotEqual(self.deck.all_cards, original_order)

    def test_deck_deal(self):
        # Verifique se a função deal remove uma carta do deck
        initial_length = len(self.deck.all_cards)
        self.deck.deal()
        self.assertEqual(len(self.deck.all_cards), initial_length - 1)

    def test_card_creation(self):
        # Verifique se a criação de uma carta funciona corretamente
        card = Card("Hearts", "A")
        self.assertEqual(card.suit, "Hearts")
        self.assertEqual(card.rank, "A")

if __name__ == "__main__":
    unittest.main()

Overwriting programs/08-Milestone-Project-2/test_Deck_class.py


In [75]:
! python3 "programs/08-Milestone-Project-2/test_Deck_class.py"

....
----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK


## Player/Dealer class

In [22]:
%%writefile "programs/08-Milestone-Project-2/Player_Dealer_class.py"
# imports
from Deck_class import *

class Hand:
    def __init__(self):
        self.cards = [] 
        self.value = 0 
        # attribute to keep track of number of aces
        self.aces = 0 

    def add_cards(self, card):
        self.value += values[card.rank]
        self.cards.append(card)
        if card.rank == "Ace":
            self.aces += 1

    def adjust_for_ace(self):
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1

    def show_hand(self, all_cards = True):
        if all_cards:
            card_list = [str(card) for card in self.cards]
            return f"The Player has in the hand {', '.join(card_list)} with a total value of {self.value}."
        else:
            card_list = [str(card) for card in self.cards][1:]
            card_value = sum([values[card.rank] for card in self.cards][1:])
            return f"The Dealer has the faceup cards {', '.join(card_list)} (value {card_value}) and a hidden card."

    def clear_hand(self):
        self.cards = []
        self.value = 0
        self.aces = 0

class Player(Hand):
    def __init__(self, total_chips=100):
        super().__init__()
        self.total_chips = total_chips
        self.current_bet = 0

    def bet(self, value_of_bet):
        self.current_bet = value_of_bet

    def win_bet(self):
        self.total_chips += 2  *self.current_bet
        self.current_bet = 0

    def lose_bet(self):
        self.total_chips -= self.current_bet
        self.current_bet = 0

    def show_chips(self):
        return f"The Player has {self.total_chips} chips"

class Dealer(Hand):
    def __init__(self):
        super().__init__()

    def __str__(self):
        card_list = [str(card) for card in self.cards]
        return f"The Dealer has in the hand {card_list} with sums {self.value}"


Overwriting programs/08-Milestone-Project-2/Player_Dealer_class.py


### Test Player Dealer class

In [34]:
%%writefile "programs/08-Milestone-Project-2/test_Player_Dealer_class.py"
# imports
import unittest
from Player_Dealer_class import *
from Deck_class import *


class TestPlayer(unittest.TestCase):
    def setUp(self):
        self.player = Player(total_chips=100)

    def test_win_bet(self):
        self.player.bet(20)
        self.player.win_bet()
        self.assertEqual(self.player.total_chips, 140)  # 100 + 2 * 20

    def test_lose_bet(self):
        self.player.bet(30)
        self.player.lose_bet()
        self.assertEqual(self.player.total_chips, 70)  # 100 - 30

    def test_str(self):
        self.assertEqual(str(self.player), "The Player has 100 chips")
        self.player.add_cards(Card("Hearts", "Ace"))
        self.assertEqual(str(self.player.show_hand()), "The Player has in the hand Ace of Hearts with a total value of 11.")
        
    

class TestHandOfPlayer(unittest.TestCase):
    def setUp(self):
        self.player = Player(total_chips=100)

    def test_add_cards(self):
        self.player.add_cards(Card("Hearts", "Ace"))
        self.assertEqual(self.player.value, 11)
        self.assertEqual(len(self.player.cards), 1)

    def test_adjust_for_ace(self):
        self.player.add_cards(Card("Hearts", "Ace"))
        self.player.add_cards(Card("Diamonds", "Ten"))
        self.player.adjust_for_ace()
        self.assertEqual(self.player.value, 21)
        self.assertEqual(len(self.player.cards), 2)

    def test_clear_hand(self):
        self.player.add_cards(Card("Spades", "King"))
        self.player.clear_hand()
        self.assertEqual(len(self.player.cards), 0)
        self.assertEqual(self.player.value, 0)

class TestDealer(unittest.TestCase):
    def setUp(self):
        self.dealer = Dealer()

    def test_str(self):
        self.dealer.add_cards(Card("Hearts", "Ace"))
        self.assertEqual(str(self.dealer), "The Dealer has in the hand ['Ace of Hearts'] with sums 11")
        self.dealer.add_cards(Card("Diamonds", "Ten"))
        self.assertEqual(len(self.dealer.cards), 2)
        self.assertEqual(str(self.dealer.show_hand(False)), "The Dealer has the faceup cards Ten of Diamonds.")

class TestHandOfDealer(unittest.TestCase):
    def setUp(self):
        self.dealer = Dealer()

    def test_add_cards(self):
        self.dealer.add_cards(Card("Hearts", "Ace"))
        self.assertEqual(self.dealer.value, 11)
        self.assertEqual(len(self.dealer.cards), 1)

    def test_adjust_for_ace(self):
        self.dealer.add_cards(Card("Hearts", "Ace"))
        self.dealer.add_cards(Card("Diamonds", "Ten"))
        self.dealer.adjust_for_ace()
        self.assertEqual(self.dealer.value, 21)
        self.assertEqual(len(self.dealer.cards), 2)

    def test_clear_hand(self):
        self.dealer.add_cards(Card("Spades", "King"))
        self.dealer.clear_hand()
        self.assertEqual(len(self.dealer.cards), 0)
        self.assertEqual(self.dealer.value, 0)

if __name__ == "__main__":
    unittest.main()


Overwriting programs/08-Milestone-Project-2/test_Player_Dealer_class.py


In [179]:
! python3 "programs/08-Milestone-Project-2/test_Player_Dealer_class.py"

..........
----------------------------------------------------------------------
Ran 10 tests in 0.000s

OK


## Table class

In [25]:
%%writefile "programs/08-Milestone-Project-2/Table_class.py"
# imports
from Player_Dealer_class import *

class Table():
    def __init__(self):
        self.player = Player()
        self.dealer = Dealer()
        self.deck = Deck()

    
    def take_bet(self):
        taking_bet = True
        while taking_bet:
            try:
                user_input = int(input("Please enter your bet: "))
                if user_input > self.player.total_chips:
                    print("You do not have enough chips!")
                elif user_input <= self.player.total_chips:
                    self.player.bet(user_input)
                    taking_bet = False
            except ValueError:
                print("Looks like you did not enter an integer!")
        print(f"Ok, your bet is {self.player.current_bet}")

    def ask_to_play(self):        
        play_game = input('Are you ready to play? Enter Yes or No.')
        return play_game.lower()[0] == 'y'

    def shuffle_deck(self):
        self.deck.shuffle()

    def deal_to_player(self):
        self.player.add_cards(self.deck.deal())
        
        self.player.adjust_for_ace()
    
    def deal_to_dealer(self):
        self.dealer.add_cards(self.deck.deal())
        self.dealer.adjust_for_ace()

    # def show_table(self, all_cards = True):
    #     player.show_hand(all_cards)
    #     dealer.show_hand(all_cards)
    
    def player_wins(self):
        self.player.win_bet()
    
    def player_loses(self):
        self.player.lose_bet()

Overwriting programs/08-Milestone-Project-2/Table_class.py


### Test Table class

In [72]:
%%writefile "programs/08-Milestone-Project-2/test_Table_class.py"
# imports
import unittest
from unittest.mock import patch
from Table_class import *

class TestTable(unittest.TestCase):
    def setUp(self):
        self.table = Table()

    def test_take_bet(self):
        with patch('builtins.input', return_value='50'):    
            # Test if the player's bet is correctly set
            self.table.player.total_chips = 100
            self.table.take_bet()
            self.assertEqual(self.table.player.current_bet, 50)  # User input is 50

    def test_ask_to_play(self):
        with patch('builtins.input', return_value='yes'):   
            # Test if the user input for playing is correctly processed
            self.assertTrue(self.table.ask_to_play())  # User input is 'Yes'

    def test_shuffle_deck(self):
        # Test if the deck is shuffled
        original_order = self.table.deck.all_cards[:]
        self.table.shuffle_deck()
        self.assertNotEqual(self.table.deck.all_cards, original_order)

    def test_deal_to_player(self):
        # Test if card are dealt to the player
        self.table.deal_to_player()
        self.assertEqual(len(self.table.player.cards), 1)

    def test_deal_to_dealer(self):
        # Test if card are dealt to the dealer
        self.table.deal_to_dealer()
        self.assertEqual(len(self.table.dealer.cards), 1)

    def test_player_wins(self):
        # Test if player wins are correctly processed
        bet = 30
        initial_chips = self.table.player.total_chips
        self.table.player.current_bet = bet
        self.table.player_wins()
        self.assertEqual(self.table.player.current_bet, 0)
        self.assertEqual(self.table.player.total_chips, initial_chips + 2 * bet)

    def test_player_loses(self):
        # Test if player losses are correctly processed
        bet = 30
        initial_chips = self.table.player.total_chips
        self.table.player.current_bet = bet
        self.table.player_loses()
        self.assertEqual(self.table.player.current_bet, 0)
        self.assertEqual(self.table.player.total_chips, initial_chips - bet)

if __name__ == '__main__':
    unittest.main()


Overwriting programs/08-Milestone-Project-2/test_Table_class.py


In [73]:
! python "programs/08-Milestone-Project-2/test_Table_class.py"

Ok, your bet is 50


.......
----------------------------------------------------------------------
Ran 7 tests in 0.001s

OK


## Game logic

In [16]:
%%writefile "programs/08-Milestone-Project-2/play_blackjack.py"
# imports
import os
from IPython.display import clear_output
from Player_Dealer_class import *
from Deck_class import *
from Table_class import *

def clear_screen():
    '''
    Function that clears the output, uses different function depending on the environment
    '''
    try:
        # Check if running in IPython environment
        ipython = get_ipython()
        if ipython is not None:
            # Execute in IPython environment
            clear_output(wait=True)
    except NameError:
        # Execute in Python terminal
        os.system('cls' if os.name == 'nt' else 'clear')

if __name__ == '__main__':

    clear_screen()
    
    # Game 
    
    # global variable
    playing = True

    # Intro message
    print("This is a game of Blackjack.")
    # Create table object
    table = Table()
    # Shuffle deck
    table.shuffle_deck()
    while playing:
        while True:
            # Ask the player if he is going to play
            if not table.ask_to_play():
                print("Bye bye! Thank you for playing!")
                playing = False
                break
            
            # Clear player and dealer hands
            table.player.clear_hand()
            table.dealer.clear_hand()

            # Clear terminal
            clear_screen()

            # Print player number of chips
            print(table.player.show_chips())

            table.take_bet()

            # Deal two cards to player
            table.deal_to_player()
            table.deal_to_player()

            # Deal two cards to dealer
            table.deal_to_dealer()
            table.deal_to_dealer()

            # Show all cards of player
            print(table.player.show_hand(all_cards = True))

            # Show cards of dealer (one is hidden)
            print(table.dealer.show_hand(all_cards = False))

            # Prompt for Player to Hit or Stand
            while True:
                player_input = input("Would you like to Hit or Stand? Enter 'h' or 's' ")
                if player_input[0].lower() == 'h':
                    table.deal_to_player()
                    print("New hand of player:")
                    print(table.player.show_hand(all_cards = True))
                    if table.player.value > 21:
                        break

                elif player_input[0].lower() == 's':
                    print("Player stands. Dealer is playing.")
                    break

                else:
                    print("Sorry, please try again.")
                    continue
            

            # If player's hand exceeds 21, run player_busts() and break out of loop
            if table.player.value > 21:
                print(f"Sorry! Player busted with {table.player.value}")
                table.player_loses()
            # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
        
            while table.dealer.value < 17:
                table.deal_to_dealer()

            # Show cards of dealer (one is hidden)
            print(table.dealer)

            # Run different winning scenarios

            if table.dealer.value > 21:
                print(f"Dealer busted! Player wins!")
                table.player_wins()

            elif table.dealer.value > table.player.value:
                print(f"Sorry! Dealer wins with {table.dealer.value}")
                table.player_loses()

            elif table.dealer.value < table.player.value:
                print(f"Player wins with {table.player.value}")
                table.player_wins()

            else:
                print(f"Dealer and player tie with {table.dealer.value}! PUSH!!")

            # Print player number of chips
            if table.player.total_chips == 0:
                print("Sorry! You lost all your chips. You can't play without chips.")
                playing = False
                break
            else:
                print(table.player.show_chips())

Overwriting programs/08-Milestone-Project-2/play_blackjack.py


In [23]:
# Play the game
%run "programs/08-Milestone-Project-2/play_blackjack.py"

The Player has 100 chips
Ok, your bet is 100
The Player has in the hand Ace of Hearts, Nine of Clubs with a total value of 20.
The Dealer has the faceup cards Ace of Diamonds (value 11) and a hidden card.
Player stands. Dealer is playing.
The Dealer has in the hand ['Eight of Clubs', 'Ace of Diamonds'] with sums 19
Player wins with 20
The Player has 300 chips
Bye bye! Thank you for playing!
